# K-nearest neighbor

In [1]:
# Data import
from preprocessing.preprocessing import download_brfss_dataset
#download_brfss_dataset("username", "token") -> insert kaggle username and api-token

   GenHealth  PhysHealth  MentHealth  Healthcare  MedCost  Checkup  HighBP  \
0        5.0        15.0        18.0         1.0      2.0      1.0     1.0   
1        3.0        88.0        88.0         2.0      1.0      4.0     3.0   
2        4.0        15.0        88.0         1.0      2.0      1.0     3.0   
3        5.0        30.0        30.0         1.0      1.0      1.0     1.0   
4        5.0        20.0        88.0         1.0      2.0      1.0     3.0   

   HighChol  HeartAttack  AngiCoro  Stroke  Asthma  Arthritis  Kidney  Sex  \
0       1.0          2.0       2.0     2.0     1.0        1.0     2.0  2.0   
1       2.0          2.0       2.0     2.0     2.0        2.0     2.0  2.0   
2       1.0          7.0       2.0     1.0     2.0        1.0     2.0  2.0   
3       1.0          2.0       2.0     2.0     2.0        1.0     2.0  2.0   
4       2.0          2.0       2.0     2.0     2.0        1.0     2.0  2.0   

   Income  SodiumSalt   Age  Height   Weight  BMI  Education  

In [2]:
#imports

from preprocessing.preprocessing import get_preprocessed_brfss_train_test_split_one_hot_encoded, get_preprocessed_brfss_train_test_split, get_preprocessed_brfss_dataset

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV


The following cell loads the already preprocessed data and makes it available for this notebook.

In [3]:
train_dataset, target = get_preprocessed_brfss_dataset()
target = target.squeeze()

The following cell loads the already preprocessed data and makes it available for this notebook.
The data is thereby split in train and test data as well as data and target.

In [4]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_train_test_split()
data_train = data_train.head(5000)
target_train = target_train.head(5000)

No we can apply a K-nearest neighbor classifier to the train data and test against the test data onhow it performs by using the accuracy score.

In [5]:
for n_neighbors in range(1,10):
    knn_estimator = KNeighborsClassifier(n_neighbors)
    knn_estimator.fit(data_train, target_train)
    diabetes_test_prediction = knn_estimator.predict(data_test)
    print("k= {} acc: {}".format(n_neighbors, accuracy_score(target_test, diabetes_test_prediction)))

/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


k= 1 acc: 0.7534262090837014


/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


k= 2 acc: 0.7054479556008608


/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


...

k= 7 acc: 0.8278853777324725


/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


k= 8 acc: 0.8262657152565409


/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


k= 9 acc: 0.8328576282704723


We can see that k=9 performs the best. However, the accuracy score is not sufficient given that the data is unbalanced at the amount of no diabetes entities is around 84% (???).

To check whether another metric would perform better, we use a grid search for hyperparameter tuning.

In [6]:
knn_estimator = KNeighborsClassifier(7)
knn_estimator.get_params()

params = {
    'n_neighbors': range (7,15),
    'metric': ('minkowski', 'cosine', 'sqeuclidean', 'manhattan')
}

grid_search_estimator = GridSearchCV(knn_estimator, params, scoring='accuracy', cv=5, return_train_score=False)
grid_search_estimator.fit(data_train,target_train.values.ravel())

results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001466,0.000124,0.032474,0.002563,minkowski,7,"{'metric': 'minkowski', 'n_neighbors': 7}",0.837,0.823,0.830,0.832,0.836,0.8316,0.005004,26
1,0.001519,0.000211,0.030041,0.001756,minkowski,8,"{'metric': 'minkowski', 'n_neighbors': 8}",0.834,0.822,0.836,0.827,0.833,0.8304,0.005161,29
2,0.001333,0.000033,0.029558,0.000664,minkowski,9,"{'metric': 'minkowski', 'n_neighbors': 9}",0.841,0.831,0.835,0.831,0.838,0.8352,0.003919,21
3,0.001358,0.000018,0.028867,0.000781,minkowski,10,"{'metric': 'minkowski', 'n_neighbors': 10}",0.842,0.832,0.832,0.833,0.837,0.8352,0.003868,21
4,0.001331,0.000029,0.030069,0.001388,minkowski,11,"{'metric': 'minkowski', 'n_neighbors': 11}",0.842,0.834,0.829,0.837,0.838,0.8360,0.004336,13
5,0.001352,0.000066,0.030006,0.000859,minkowski,12,"{'metric': 'minkowski', 'n_neighbors': 12}",0.840,0.834,0.828,0.839,0.835,0.8352,0.004261,19
6,0.001299,0.000024,0.029410,0.001454,minkowski,13,"{'metric': 'minkowski', 'n_neighbors': 13}",0.835,0.834,0.833,0.840,0.839,0.8362,0.002786,11
7,0.001319,0.000015,0.030628,0.002135,minkowski,14,"{'metric': 'minkowski', 'n_neighbors': 14}",0.838,0.836,0.833,0.840,0.840,0.8374,0.002653,6
8,0.001292,0.000033,0.069897,0.010596,cosine,7,"{'metric': 'cosine', 'n_neighbors': 7}",0.821,0.829,0.822,0.832,0.845,0.8298,0.008658,32
9,0.001282,0.000024,0.065062,0.003918,cosine,8,"{'metric': 'cosine', 'n_neighbors': 8}",0.829,0.828,0.828,0.831,0.836,0.8304,0.003007,29


best score is 0.841 with params {'metric': 'manhattan', 'n_neighbors': 14}


Unfortunately the best combination only leads to an increase of 0.0046 in accuracy which is still not satisfying.

Next we try to balance the data using oversampling.

In [7]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_train_test_split(True)
data_train = data_train.head(5000)
target_train = target_train.head(5000)

In [8]:
knn_estimator = KNeighborsClassifier()
knn_estimator.get_params()

params = {
    'n_neighbors': range (1,15),
    'metric': ('euclidean', 'minkowski', 'cosine', 'sqeuclidean', 'manhattan')
}

grid_search_estimator = GridSearchCV(knn_estimator, params, scoring='accuracy', cv=5, return_train_score=False)
grid_search_estimator.fit(data_train,target_train.values.ravel())

results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)

print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001495,0.000118,0.031480,0.005082,euclidean,1,"{'metric': 'euclidean', 'n_neighbors': 1}",0.517,0.494,0.490,0.498,0.476,0.4950,0.013266,3
1,0.001454,0.000075,0.029023,0.000479,euclidean,2,"{'metric': 'euclidean', 'n_neighbors': 2}",0.440,0.437,0.426,0.422,0.410,0.4270,0.010807,12
2,0.001461,0.000087,0.027639,0.001011,euclidean,3,"{'metric': 'euclidean', 'n_neighbors': 3}",0.432,0.428,0.415,0.413,0.426,0.4228,0.007467,14
3,0.001388,0.000016,0.028204,0.000791,euclidean,4,"{'metric': 'euclidean', 'n_neighbors': 4}",0.424,0.408,0.420,0.401,0.406,0.4118,0.008727,21
4,0.001389,0.000033,0.028685,0.001399,euclidean,5,"{'metric': 'euclidean', 'n_neighbors': 5}",0.406,0.397,0.390,0.379,0.396,0.3936,0.008913,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,0.001312,0.000026,0.043134,0.001123,manhattan,10,"{'metric': 'manhattan', 'n_neighbors': 10}",0.380,0.385,0.376,0.365,0.370,0.3752,0.007082,42
66,0.001319,0.000027,0.041874,0.001264,manhattan,11,"{'metric': 'manhattan', 'n_neighbors': 11}",0.360,0.379,0.373,0.361,0.354,0.3654,0.009178,52
67,0.001403,0.000097,0.044775,0.001366,manhattan,12,"{'metric': 'manhattan', 'n_neighbors': 12}",0.365,0.395,0.359,0.351,0.356,0.3652,0.015574,53
68,0.001333,0.000036,0.045055,0.002396,manhattan,13,"{'metric': 'manhattan', 'n_neighbors': 13}",0.358,0.367,0.346,0.343,0.359,0.3546,0.008868,61


best score is 0.5132000000000001 with params {'metric': 'cosine', 'n_neighbors': 1}


We can see that oversampling doesn't improve the classification.